# Bin profiles

## Import packages

In [7]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

from datetime import datetime
from dateutil.relativedelta import relativedelta

## Import fetch data notebook

In [2]:
# fetch_data_notebook = 'lmm_fetch_data.ipynb'

# %run {fetch_data_notebook}

# plot_data_notebook = 'lmm_plot_data.ipynb'

# %run {plot_data_notebook}

fetch_data_notebook = 'fetch_argo.ipynb'

%run {fetch_data_notebook}

## Import data binning notebook

In [3]:
data_binner_notebook = 'lmm_data_binner.ipynb'

%run {data_binner_notebook}

## Retrieve data from Argovis

In [4]:
# start_date = '2020-1-01'
# end_date = '2020-01-03'
# pressure_range = '[0,500]'
# region = 'atlantic_coords'
# argo_source = 'argo_core'

# fetch_obj = ArgovisDataImporter(start_date, end_date, pressure_range, source=argo_source, region=region)

# df = fetch_obj.get_cleaned_profile()

# len(df)

In [29]:
atlantic_coords = [[-40.078125,29.840644],[-33.368671,30.338837],[-26.614528,30.492027],
                    [-19.863281,30.297018],[-20.039063,-30.145127],[-26.724822,-30.384017],
                    [-33.419918,-30.281826],[-40.078125,-29.840644],[-40.078125,29.840644]]
presRange ='[0,500]'
shape = str(atlantic_coords)
#URL_PREFIX = 'https://argovis-api.colorado.edu'
startDate = '2021-04-01T00:00:00Z'
endDate   = '2021-05-31T00:00:00Z'

start_date = datetime.strptime('2021-01-01T00:00:00', '%Y-%m-%dT%H:%M:%S')

date_range = []

for i in np.arange(1,13):
    
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    
    date_range.append((start_date, end_date))
    
    start_date = end_date + relativedelta(days=1)

# Convert back to string
date_range_str = []
for i in date_range:
    
    start = i[0]
    end = i[1]
    
    start = start.strftime("%Y-%m-%dT%H:%M:%SZ")
    end = end.strftime("%Y-%m-%dT%H:%M:%SZ")
    
    i_range = (start, end)
    
    date_range_str.append(i_range)
    
print(date_range_str)

[('2021-01-01T00:00:00Z', '2021-01-31T00:00:00Z'), ('2021-02-01T00:00:00Z', '2021-02-28T00:00:00Z'), ('2021-03-01T00:00:00Z', '2021-03-31T00:00:00Z'), ('2021-04-01T00:00:00Z', '2021-04-30T00:00:00Z'), ('2021-05-01T00:00:00Z', '2021-05-31T00:00:00Z'), ('2021-06-01T00:00:00Z', '2021-06-30T00:00:00Z'), ('2021-07-01T00:00:00Z', '2021-07-31T00:00:00Z'), ('2021-08-01T00:00:00Z', '2021-08-31T00:00:00Z'), ('2021-09-01T00:00:00Z', '2021-09-30T00:00:00Z'), ('2021-10-01T00:00:00Z', '2021-10-31T00:00:00Z'), ('2021-11-01T00:00:00Z', '2021-11-30T00:00:00Z'), ('2021-12-01T00:00:00Z', '2021-12-31T00:00:00Z')]


In [ ]:
df = loop_fetch(shape, presRange, date_range_str)

Number of items: 623
https://argovis-api.colorado.edu/profiles?&startDate=2021-01-01T00:00:00Z&endDate=2021-01-31T23:59:59Z&polygon=[[-40.078125, 29.840644], [-33.368671, 30.338837], [-26.614528, 30.492027], [-19.863281, 30.297018], [-20.039063, -30.145127], [-26.724822, -30.384017], [-33.419918, -30.281826], [-40.078125, -29.840644], [-40.078125, 29.840644]]&data=psal,temp,psal_argoqc,pres_argoqc,temp_argoqc&source=argo_core
Number of items: 551
https://argovis-api.colorado.edu/profiles?&startDate=2021-02-01T00:00:00Z&endDate=2021-02-28T23:59:59Z&polygon=[[-40.078125, 29.840644], [-33.368671, 30.338837], [-26.614528, 30.492027], [-19.863281, 30.297018], [-20.039063, -30.145127], [-26.724822, -30.384017], [-33.419918, -30.281826], [-40.078125, -29.840644], [-40.078125, 29.840644]]&data=psal,temp,psal_argoqc,pres_argoqc,temp_argoqc&source=argo_core
Number of items: 644
https://argovis-api.colorado.edu/profiles?&startDate=2021-03-01T00:00:00Z&endDate=2021-03-31T23:59:59Z&polygon=[[-40.07

In [ ]:
print(len(df))
print(df)
print(df.columns)

## Save dataframe

In [ ]:
df.to_csv('2021_argo_core_dataframe.csv')

In [ ]:
print(df.columns)

profile_vars = ['_id', 'geolocation', 'timestamp', 'source_info', 'platform_type', 'data']

df_subset = df[profile_vars]
df_subset.head()

In [ ]:
df_subset = expand_lon_lat(df_subset)

print(df_subset.columns)

In [ ]:
df_subset = expand_source_info(df_subset)

print(df_subset.columns)



In [ ]:
print(df_subset.columns)

parsed_df = lmm_parse_df(df_subset)

In [ ]:
parsed_df.head()

In [ ]:
# Filter to get argo_core

# Gives error. Maybe because not finding argo_core string? why?

#df = parsed_df.loc[parsed_df['source'].str.contains("argo_core")]



In [ ]:
# map column names so argo and goship the same

name_mapping = {}
name_mapping['psal_argoqc'] = 'psal_qc'
name_mapping['temp_argoqc'] = 'temp_qc'
name_mapping['pres_argoqc'] = 'pres_qc'

df = parsed_df.rename(columns=name_mapping)

print(df.columns)


In [ ]:
class ShowProgressContextManager:
    def __init__(self, out, command=''):
        self.out = out
        self.command = command
        
    def __enter__(self):
        css_style = "<style>span.start { margin:0; padding: 0; color: blue; }</style>"
        html_str = f'<span class="start">Running {self.command}...</span>'
       
        with self.out:
            display(HTML(css_style))
            display(HTML(html_str))
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        
        css_style = "<style>span.end { margin: 0; padding: 0; color: green; }</style>"
        html_str = '<span class="end">Finished</span>'
        
        with self.out:
            display(HTML(css_style)) 
            display(HTML(html_str))

## Create xarray dataset binned on pressure


In [ ]:
pressure_range = [0,500]
pressure_bin_size = 5

binner_obj = DataBinner(df, pressure_range, pressure_bin_size)

out = widgets.Output()

message = 'Create xarray dataset'

with ShowProgressContextManager(out, message):
    
    ds_all = binner_obj.create_xarray()

display(out)


## Create xarray dataset without binning on pressure
Make pressure a data variable, and not a coordinate

Use index only as the coordinate. Make sure to reindex df so it is all unique

### Look at dataset created

In [ ]:
print(ds_all)
print(list(ds_all.coords))
print(list(ds_all.keys()))


## Save dataset to netcdf

Since limited on the number of data to download, consider saving to files and then combining when reading them all in

In [ ]:
ds_all.to_netcdf('temp_psal_dataset.nc')

## Example queries

In [ ]:
xr.plot.scatter(ds_all,'pres', 'temp', size=8)

# TODO

Can't bin on time yet since only have the timestamp variable

### Select all values where day = 2

In [ ]:
a = ds_all.where(ds_all.day == 2, drop=True)
print(a)

In [ ]:
xr.plot.scatter(a,'pres', 'temp', size=8)

In [ ]:
ds_all_grouped = binner_obj.bin_on_time(ds_all, 'day')

In [ ]:

for name, group in ds_all_grouped:
    pres = group['pres'].values
    temp = group['temp'].data
    psal = group['psal'].data
    plt.plot(pres,temp)
